In [1]:
import os
import tensorflow as tf

use_cpu = False

if use_cpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    os.environ['CPU_ONLY'] = "TRUE"

    physical_devices = tf.config.list_physical_devices('CPU')

    tf.config.set_logical_device_configuration(
        physical_devices[0],
        [tf.config.LogicalDeviceConfiguration() for i in range(8)])
    logical_devices = tf.config.list_logical_devices('CPU')

    print(logical_devices)
else:
    os.environ['CPU_ONLY'] = "FALSE"
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from latentneural.data import DataManager
from latentneural.lfads import LFADS
from latentneural.lfads.train import train
from latentneural.lfads.adaptive_weights import AdaptiveWeights


data_dir = os.path.join('..', '..', 'latentneural', 'data', 'storage', 'lorenz', '20210610T215300')
dataset, settings = DataManager.load_dataset(
    directory=data_dir,
    filename='dataset.h5')

In [3]:
neural_data = dataset['train_data'].astype('float')
valid_neural_data = dataset['valid_data'].astype('float')

In [4]:
from datetime import datetime
logdir = os.path.join(data_dir, 'results_new_lfads', datetime.now().strftime("%Y%m%d-%H%M%S"))

initializer=tf.keras.initializers.VarianceScaling(
    scale=1.0, mode='fan_in', distribution='normal'
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-2,
    beta_1=0.9, 
    beta_2=0.999,
    epsilon=1e-01)

layer_settings=dict(
    kernel_initializer=initializer,
    kernel_regularizer=tf.keras.regularizers.l2(l=0.1)
)

model = train(
    adaptive_lr=dict(factor=0.95, patience=10, min_lr=1e-5),
    model_settings=dict(
        factors_space=3,
        encoded_space=64,
        max_grad_norm=200,
        encoded_var_max=0.1,
        encoded_var_min=0.1,
        original_generator=False,
        timestep=settings['step'],
        encoder_dense=dict(
            layer_settings),
        dense=layer_settings,
        dense_mean=layer_settings,
        dense_logvar=layer_settings,
        neural_dense=layer_settings,
        encoder=dict(
            layer_settings,
            dropout=0.05),
        decoder=dict(
            kernel_initializer=initializer,
            kernel_regularizer=tf.keras.regularizers.l2(l=1),
            recurrent_regularizer=tf.keras.regularizers.l2(l=1)
        ),
    ), 
    optimizer=optimizer, 
    epochs=2000, 
    logdir=logdir,
    train_dataset=neural_data, 
    val_dataset=valid_neural_data,
    adaptive_weights=AdaptiveWeights(
        initial=[1.0, 0.0, 0.0],
        update_starts=[0, 1000, 0],
        update_rates=[0., 0.0005, 0.0005],
        min_weight=[1.0, 0.0, 0.0]
    ),
    batch_size=16
)

/home/alessandrofacchin/miniconda/envs/latentneural/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/2000
5/5 [==============================] - 3s 398ms/step - loss: 1815.9565 - loss/loglike: 1832.5340 - loss/kldiv: 48.1224 - loss/reg: 1085.2428 - weights/loglike: 1.0000 - weights/kldiv: 0.0000e+00 - weights/reg: 5.8333e-04 - learning_rate: 0.0100 - grads/EncoderLSTMForward_lstmcell2: 21.0634 - grads/EncoderLSTMBackward_lstmcell1: 22.6101 - grads/EncoderDense: 3732.5476 - grads/DenseMean: 18.6132 - grads/DecoderGRU_grucell: 302.0012 - grads/Dense: 106.3872 - grads/NeuralDense: 142.4070 - norms/EncoderLSTMForward_lstmcell2: 1958.0798 - norms/EncoderLSTMBackward_lstmcell1: 1975.8746 - norms/EncoderDense: 5937.7233 - norms/DenseMean: 412.5640 - norms/DecoderGRU_grucell: 1985.1325 - norms/Dense: 18.5675 - norms/NeuralDense: 40.8503 - val_loss: 1693.4352 - val_loss/loglike: 1693.4352 - val_loss/kldiv: 48.2408
Epoch 2/2000
5/5 [==============================] - 1s 161ms/step - loss: 1638.7985 - loss/loglike: 1654.4707 - loss/kldiv: 48.3242 - loss/reg: 1085.5870 - weights/loglike: 1

5/5 [==============================] - 1s 149ms/step - loss: 808.2296 - loss/loglike: 802.1141 - loss/kldiv: 55.1999 - loss/reg: 1085.5158 - weights/loglike: 1.0000 - weights/kldiv: 0.0000e+00 - weights/reg: 0.0231 - learning_rate: 0.0100 - grads/EncoderLSTMForward_lstmcell2: 676.4637 - grads/EncoderLSTMBackward_lstmcell1: 695.2514 - grads/EncoderDense: 91227.4648 - grads/DenseMean: 906.9953 - grads/DecoderGRU_grucell: 13395.2521 - grads/Dense: 5407.9093 - grads/NeuralDense: 12939.5177 - norms/EncoderLSTMForward_lstmcell2: 38429.8311 - norms/EncoderLSTMBackward_lstmcell1: 38778.1309 - norms/EncoderDense: 116838.7604 - norms/DenseMean: 8105.0268 - norms/DecoderGRU_grucell: 38939.1748 - norms/Dense: 356.0308 - norms/NeuralDense: 839.8781 - val_loss: 795.1667 - val_loss/loglike: 795.1667 - val_loss/kldiv: 56.6811
Epoch 11/2000
5/5 [==============================] - 1s 161ms/step - loss: 778.2887 - loss/loglike: 781.0304 - loss/kldiv: 58.9037 - loss/reg: 1085.3674 - weights/loglike: 1.0000

5/5 [==============================] - 1s 157ms/step - loss: 749.5790 - loss/loglike: 746.4449 - loss/kldiv: 104.7901 - loss/reg: 1076.8879 - weights/loglike: 1.0000 - weights/kldiv: 0.0000e+00 - weights/reg: 0.0456 - learning_rate: 0.0100 - grads/EncoderLSTMForward_lstmcell2: 2004.2378 - grads/EncoderLSTMBackward_lstmcell1: 2021.6804 - grads/EncoderDense: 194822.9870 - grads/DenseMean: 3796.7698 - grads/DecoderGRU_grucell: 34679.7839 - grads/Dense: 16734.2469 - grads/NeuralDense: 19821.9818 - norms/EncoderLSTMForward_lstmcell2: 74925.0358 - norms/EncoderLSTMBackward_lstmcell1: 75599.2539 - norms/EncoderDense: 229926.4609 - norms/DenseMean: 15882.5480 - norms/DecoderGRU_grucell: 75767.3346 - norms/Dense: 640.2539 - norms/NeuralDense: 1704.0128 - val_loss: 769.5743 - val_loss/loglike: 769.5743 - val_loss/kldiv: 88.9845
Epoch 20/2000
5/5 [==============================] - 1s 154ms/step - loss: 748.4741 - loss/loglike: 748.8791 - loss/kldiv: 92.9315 - loss/reg: 1074.8381 - weights/loglike

5/5 [==============================] - 1s 150ms/step - loss: 747.2679 - loss/loglike: 750.1428 - loss/kldiv: 107.9534 - loss/reg: 1042.2341 - weights/loglike: 1.0000 - weights/kldiv: 0.0000e+00 - weights/reg: 0.0681 - learning_rate: 0.0095 - grads/EncoderLSTMForward_lstmcell2: 3741.2303 - grads/EncoderLSTMBackward_lstmcell1: 3762.5280 - grads/EncoderDense: 289020.7135 - grads/DenseMean: 6505.0203 - grads/DecoderGRU_grucell: 53915.8411 - grads/Dense: 27818.1813 - grads/NeuralDense: 25949.4422 - norms/EncoderLSTMForward_lstmcell2: 111439.7031 - norms/EncoderLSTMBackward_lstmcell1: 112427.5182 - norms/EncoderDense: 346540.7891 - norms/DenseMean: 23788.7607 - norms/DecoderGRU_grucell: 112135.9596 - norms/Dense: 894.0797 - norms/NeuralDense: 2575.3283 - val_loss: 767.8731 - val_loss/loglike: 767.8731 - val_loss/kldiv: 94.5844
Epoch 29/2000
5/5 [==============================] - 1s 146ms/step - loss: 746.9858 - loss/loglike: 739.1647 - loss/kldiv: 114.4352 - loss/reg: 1035.0025 - weights/log

...

5/5 [==============================] - 1s 155ms/step - loss: 753.5928 - loss/loglike: 689.0692 - loss/kldiv: 56.3630 - loss/reg: 211.3584 - weights/loglike: 1.0000 - weights/kldiv: 1.0000 - weights/reg: 1.0000 - learning_rate: 1.0000e-05 - grads/EncoderLSTMForward_lstmcell2: 2087243.1875 - grads/EncoderLSTMBackward_lstmcell1: 2195717.5417 - grads/EncoderDense: 55077680.3333 - grads/DenseMean: 3567687.5833 - grads/DecoderGRU_grucell: 19192677.6667 - grads/Dense: 4576389.6250 - grads/NeuralDense: 926143.3125 - norms/EncoderLSTMForward_lstmcell2: 5263726.0833 - norms/EncoderLSTMBackward_lstmcell1: 5308344.5417 - norms/EncoderDense: 27351345.5000 - norms/DenseMean: 1257206.6875 - norms/DecoderGRU_grucell: 2134764.2917 - norms/Dense: 49093.0765 - norms/NeuralDense: 161434.1628 - val_loss: 758.7402 - val_loss/loglike: 706.2463 - val_loss/kldiv: 52.4939
Epoch 1962/2000
5/5 [==============================] - 1s 178ms/step - loss: 753.2386 - loss/loglike: 691.5442 - loss/kldiv: 56.5243 - loss/r

5/5 [==============================] - 1s 163ms/step - loss: 758.1198 - loss/loglike: 701.6808 - loss/kldiv: 56.7363 - loss/reg: 211.3338 - weights/loglike: 1.0000 - weights/kldiv: 1.0000 - weights/reg: 1.0000 - learning_rate: 1.0000e-05 - grads/EncoderLSTMForward_lstmcell2: 2097385.5417 - grads/EncoderLSTMBackward_lstmcell1: 2206344.6250 - grads/EncoderDense: 55353533.6667 - grads/DenseMean: 3585835.6667 - grads/DecoderGRU_grucell: 19296411.6667 - grads/Dense: 4598597.9583 - grads/NeuralDense: 929992.3438 - norms/EncoderLSTMForward_lstmcell2: 5285722.2083 - norms/EncoderLSTMBackward_lstmcell1: 5330526.9167 - norms/EncoderDense: 27461605.5000 - norms/DenseMean: 1262235.4375 - norms/DecoderGRU_grucell: 2142745.1042 - norms/Dense: 49309.7376 - norms/NeuralDense: 162161.1198 - val_loss: 758.7729 - val_loss/loglike: 706.2854 - val_loss/kldiv: 52.4875
Epoch 1971/2000
5/5 [==============================] - 1s 181ms/step - loss: 753.9245 - loss/loglike: 695.1726 - loss/kldiv: 56.3129 - loss/r

5/5 [==============================] - 1s 179ms/step - loss: 757.7067 - loss/loglike: 696.4175 - loss/kldiv: 56.3100 - loss/reg: 211.3254 - weights/loglike: 1.0000 - weights/kldiv: 1.0000 - weights/reg: 1.0000 - learning_rate: 1.0000e-05 - grads/EncoderLSTMForward_lstmcell2: 2108406.0625 - grads/EncoderLSTMBackward_lstmcell1: 2217938.3958 - grads/EncoderDense: 55653411.3333 - grads/DenseMean: 3604518.7500 - grads/DecoderGRU_grucell: 19397912.5000 - grads/Dense: 4620812.2083 - grads/NeuralDense: 933938.6250 - norms/EncoderLSTMForward_lstmcell2: 5307718.3333 - norms/EncoderLSTMBackward_lstmcell1: 5352709.2917 - norms/EncoderDense: 27571865.5000 - norms/DenseMean: 1267264.1875 - norms/DecoderGRU_grucell: 2150725.9167 - norms/Dense: 49526.3988 - norms/NeuralDense: 162888.0768 - val_loss: 758.8126 - val_loss/loglike: 706.3211 - val_loss/kldiv: 52.4915
Epoch 1980/2000
5/5 [==============================] - 1s 182ms/step - loss: 753.7947 - loss/loglike: 695.7304 - loss/kldiv: 56.4381 - loss/r

5/5 [==============================] - 1s 163ms/step - loss: 754.4649 - loss/loglike: 694.1754 - loss/kldiv: 56.5424 - loss/reg: 211.3027 - weights/loglike: 1.0000 - weights/kldiv: 1.0000 - weights/reg: 1.0000 - learning_rate: 1.0000e-05 - grads/EncoderLSTMForward_lstmcell2: 2118146.3542 - grads/EncoderLSTMBackward_lstmcell1: 2228142.7292 - grads/EncoderDense: 55918542.0000 - grads/DenseMean: 3621754.0833 - grads/DecoderGRU_grucell: 19498675.5000 - grads/Dense: 4642777.3750 - grads/NeuralDense: 937757.1771 - norms/EncoderLSTMForward_lstmcell2: 5329714.4583 - norms/EncoderLSTMBackward_lstmcell1: 5374891.6667 - norms/EncoderDense: 27682086.6667 - norms/DenseMean: 1272292.9375 - norms/DecoderGRU_grucell: 2158706.7292 - norms/Dense: 49743.0599 - norms/NeuralDense: 163615.0339 - val_loss: 758.7654 - val_loss/loglike: 706.2819 - val_loss/kldiv: 52.4836
Epoch 1989/2000
5/5 [==============================] - 1s 172ms/step - loss: 753.5016 - loss/loglike: 695.1213 - loss/kldiv: 56.7571 - loss/r

5/5 [==============================] - 1s 164ms/step - loss: 755.5045 - loss/loglike: 691.8548 - loss/kldiv: 56.1870 - loss/reg: 211.3031 - weights/loglike: 1.0000 - weights/kldiv: 1.0000 - weights/reg: 1.0000 - learning_rate: 1.0000e-05 - grads/EncoderLSTMForward_lstmcell2: 2129021.1667 - grads/EncoderLSTMBackward_lstmcell1: 2239587.1042 - grads/EncoderDense: 56201106.6667 - grads/DenseMean: 3640199.2083 - grads/DecoderGRU_grucell: 19601271.5000 - grads/Dense: 4663789.7500 - grads/NeuralDense: 941405.1458 - norms/EncoderLSTMForward_lstmcell2: 5351710.5833 - norms/EncoderLSTMBackward_lstmcell1: 5397074.0417 - norms/EncoderDense: 27792284.3333 - norms/DenseMean: 1277321.6875 - norms/DecoderGRU_grucell: 2166687.5417 - norms/Dense: 49959.7210 - norms/NeuralDense: 164341.9909 - val_loss: 758.7440 - val_loss/loglike: 706.2502 - val_loss/kldiv: 52.4938
Epoch 1998/2000
5/5 [==============================] - 1s 161ms/step - loss: 753.2695 - loss/loglike: 690.4426 - loss/kldiv: 56.6759 - loss/r

In [12]:
import latentneural.plot as lnp
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


log_f, _, _, _ = model(neural_data.astype('float'), training=False)
fig_rates = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=np.exp(log_f[1, :, :].numpy().T), colorscale="Blues"))

fig_rates.update_layout(title='Firing Rates of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_rates_n = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=dataset['train_rates'][1, :, :].T, colorscale="Blues"))

fig_rates_n.update_layout(title='Firing Rates of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_rates.show()
fig_rates_n.show()

In [6]:
log_f, (g0, mean, logvar), z, _ = model(neural_data.astype('float'), training=False)

fig_z = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=z.numpy()[0, :, :].T, colorscale="Blues"))

fig_z.update_layout(title='Latent Variables of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_z_n = go.Figure(data=go.Heatmap(
    x=list(range(log_f.shape[1])), 
    y=list(range(log_f.shape[-1])), 
    z=dataset['train_latent'][0, :, :].T, colorscale="Blues"))

fig_z_n.update_layout(title='Latent Variables of Condition Trial #%d' % (0),
                xaxis_title='Time',
                yaxis_title='Neurons')

fig_z.show()
fig_z_n.show()

In [7]:
latent_reconstructed_unsorted = z.numpy().T
latent_original = dataset['train_latent'].T

print(latent_reconstructed_unsorted.shape)
print(latent_original.shape)

(3, 100, 70)
(3, 100, 70)


In [8]:
latent_reconstructed_unsorted_r = latent_reconstructed_unsorted.reshape(
    3, 
    latent_reconstructed_unsorted.shape[1] * latent_reconstructed_unsorted.shape[2]).T

latent_original_r = latent_original.reshape(
    3, 
    latent_original.shape[1] * latent_original.shape[2]).T

In [9]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(latent_reconstructed_unsorted_r, latent_original_r)
print('Score: ', clf.score(latent_reconstructed_unsorted_r, latent_original_r))

predictions = clf.predict(latent_reconstructed_unsorted_r).T.reshape(3, latent_reconstructed_unsorted.shape[1], latent_reconstructed_unsorted.shape[2])

Score:  0.9077030933494353


In [10]:
fig = go.Figure()
condition=0
for d in range(3):
    # Create and style traces
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=latent_original[d,:,condition], name='%d original' % (d),
                            line=dict(color=px.colors.qualitative.Plotly[d], width=10), opacity=0.2))
    fig.add_trace(go.Scatter(x=dataset['time_data'], y=predictions[d,:,condition], name='%d reconstructed' % (d),
                            line = dict(color=px.colors.qualitative.Plotly[d], width=2)))

# Edit the layout
fig.update_layout(title='Latent Trajectories of Condition #%d, Original vs Reconstructed' % (condition),
                xaxis_title='Time',
                yaxis_title='Value')
fig.show()

# Vaidation Performance

In [11]:
log_f, (g0, mean, logvar), z, _ = model(dataset['valid_data'].astype('float'), training=False)
latent_reconstructed_unsorted = z.numpy().T
latent_original = dataset['valid_latent'].T
latent_reconstructed_unsorted_r = latent_reconstructed_unsorted.reshape(
    3, latent_reconstructed_unsorted.shape[1] * latent_reconstructed_unsorted.shape[2]).T
latent_original_r = latent_original.reshape(
    3, latent_original.shape[1] * latent_original.shape[2]).T

clf = Ridge(alpha=1.0)
clf.fit(latent_reconstructed_unsorted_r, latent_original_r)
print('Score: ', clf.score(latent_reconstructed_unsorted_r, latent_original_r))

Score:  0.9026240504031989
